<a href="https://colab.research.google.com/github/hvenanc/Precificador-NFE-4.0/blob/main/Sistema_XML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [311]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dicionário de TAGS do XML

### - cProd : Código do Fornecedor
### - cEAN : EAN do Produto, em alguns casos é o EAN da Caixa do Produto
### - xProd: Nome do Produto
### - NCM: Código NCM do produto
### - vUnCom: Valor unitário do produto
### - uCom : Unidade comercializada
### - qCom : Quantidade comercializada
### - CST : Código de Situação Tributária, define quais produtos possuem substituuição tributária: CSOSN: 201 - ST Simples Nacional e CST 10 - ST Regime Normal.
### - vICMSST: Valor da ST.
### - pICMS: Alíquota ICMS
### - vIPI: Define o valor do IPI.

In [312]:
import xml.etree.ElementTree as ET
from xml.dom import minidom

In [313]:
xml = open('/content/drive/MyDrive/Python + XML/XMLS/atlas.xml')
nfe = minidom.parse(xml)

In [314]:
num_nfe = nfe.getElementsByTagName('nNF')
n_NFE = num_nfe[0].firstChild.data
print(n_NFE)

208586


In [315]:
#Funções 

#Função que varre dados do tipo text do xml e insere em uma lista de string.
def nodeText(tag,nfe):

  dados = []
  item = nfe.getElementsByTagName(tag)

  for x in item:
    dados.append(x.firstChild.data)
  
  return dados

#Função que varre dados do tipo number do xml e insere em uma lista de float.
def nodeNumber(tag,nfe):

  dados = []
  item = nfe.getElementsByTagName(tag)

  for x in item:
    dados.append(float(x.firstChild.data))

  return dados

#Função que completa com o 0 as posições do produtos sem substituição tributária

def listaST(lista1,lista2):

  a = len(lista1) - len(lista2)

  while(a != 0):
    lista2.append(0)
    a-=1

  x = 0
  y = 0
  subsTrib = []
  
  while(x < len(lista1)):
    if lista1[x] == '201' or lista1[x] == '10':
      subsTrib.append(float(lista2[y]))
      y+=1
    else:
      subsTrib.append(float(0))
    x+=1

  return subsTrib

#Criando a lista com o CST apenas da Substituição Tributátia (ST)
#Aqui o no ICMS é pai do CST

def cstList(lista):
  
  x = 0
  inter = []

  while(x<len(lista)):
    a = lista[x]
    inter.append(a)
    x+=4

  return inter

#Calcula o preço de venda

from math import floor
def precoVenda(lista,margem):

  custo = []
  venda = []
  x = 0
  while(x < len(lista)):
    y = (lista[x])/margem
    custo.append(round(y,2))
    
    j = floor(custo[x])
    y = round(j)
    if (j - y) < 1:
      j = j + (0.99 - (j - y))
      venda.append(round(j,2))
      
    x += 1

  return venda

#Ajustando IPI dos produtos

def ipiLista(lista):
  x = 1
  inter = []

  while(x<len(lista)):
    a = lista[x]
    inter.append(a)
    x+=4

  return inter

def listaIPI(lista1,lista2):

  a = len(lista1) - len(lista2)

  while(a != 0):
    lista2.append(0)
    a-=1

  x = 0
  y = 0
  ipi = []
  
  while(x < len(lista1)):
    if lista1[x] == '50':
      ipi.append(float(lista2[y]))
      y+=1
    else:
      ipi.append(float(0))
    x+=1

  return ipi


In [316]:
#Encontrando o Regime Tributário 

item = nfe.getElementsByTagName('CSOSN')

if len(item) != 0:
  codTributacao = nodeText('CSOSN',nfe)
  subs = nodeText('vICMSST',nfe)
else:
  codTributacao = nodeText('CST',nfe)
  codTributacao = cstList(codTributacao)
  subs = nodeText('vICMSST',nfe)

In [317]:
#Teste da Chamada da Função
x = nodeText('cEAN',nfe)
subs

[]

In [318]:
#Teste Chamada função do IPI
cst = nodeText('CST',nfe)
ipi = nodeNumber('vIPI',nfe)
cstIpi = ipiLista(cst)
listaIPI = listaIPI(cstIpi,ipi[:-1])
listaIPI

[10.35]

In [319]:
#Fazendo algumas verificações:

#Quantidade de itens NFE

qt_itens = len(nodeText('xProd',nfe))
qt_itens

1

In [320]:
#Criação do DataFrame

import pandas as pd

df = pd.DataFrame()
df['EAN'] = nodeText('cEAN',nfe)
df['Produto'] = nodeText('xProd',nfe)
df['Quantidade'] = nodeNumber('qCom',nfe)
df['Valor UNIT'] = nodeNumber('vUnCom',nfe)
df['NCM'] = nodeText('NCM',nfe)
valorT = nodeNumber('vProd',nfe)
df['Valor Total'] = valorT[:-1]

In [321]:
st = listaST(codTributacao,subs)
st
if qt_itens == len(st):
  df['ST'] = st
else:
  df['ST'] = 0

In [322]:
ipi = nodeNumber('vIPI',nfe)
ipi
if qt_itens == len(listaIPI):
  df['IPI'] = listaIPI
else:
  df['IPI'] = 0

In [ ]:
df['Custo Final'] = df['Valor UNIT'] + ((df['ST']+df['IPI'])/df['Quantidade'])
margem = float(input('Insira a margem de lucro: '))
df['Preço de Venda'] = precoVenda(list(df['Custo Final']),margem)
df

In [324]:
#Gerando planilha no excel

df.to_excel('precos.xlsx',sheet_name= f'NFE {n_NFE}',index = False)